In [1]:
import gymnasium as gym
import matplotlib.pyplot as plt
import keras
from keras import layers
import ale_py
from gymnasium.wrappers.atari_preprocessing import AtariPreprocessing
from gymnasium.wrappers.frame_stack import FrameStack
import numpy as np
import tensorflow as tf

gym.register_envs(ale_py)

https://gymnasium.farama.org/ Bra sida för RL tutorials (och såklart all doc)

In [12]:
env = gym.make("BreakoutNoFrameskip-v4")
env = AtariPreprocessing(env)
env = FrameStack(env, 4)

num_actions = env.action_space.n
trigger = lambda x: x % 20 == 0
# env = gym.wrappers.RecordVideo(env, video_folder="videos/", episode_trigger=trigger, disable_logger=True)

In [ ]:
def create_q_model():
    return keras.Sequential(
        [
        keras.Input(shape=(84, 84, 4)),
        layers.Conv2D(32, kernel_size=8, strides=4, activation="relu"),
        layers.Conv2D(64, kernel_size=4, strides=2, activation="relu"), 
        layers.Conv2D(63, kernel_size=3, strides=1, activation="relu"),
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dense(num_actions, activation="linear")
    ]
)

In [4]:
model = create_q_model()
model_target = create_q_model()

optimizer = keras.optimizers.Adam(learning_rate=.00025, clipnorm=1.0)

c:\KODA\ITHS\9_DeepLearning\DeepLearning\.venv\Lib\site-packages\keras\src\layers\core\lambda_layer.py:65: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
action_history = []
state_history = []
state_next_history = []
done_history = []
rewards_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0

In [6]:
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.1
epsilon_max = 1.0
epsilon_interval = epsilon_max - epsilon_min
batch_size = 32

max_episodes = 0
max_frames = 10_000_000

# antalet frames att ta random actions och observera output
epsilon_random_frames = 50_000
# antalet av frames för utforskande
epsilon_greedy_frames = 1_000_000
# Max replay buffer
max_memory_length = 100_000
max_steps_per_episode = 10_000
# Hur ofta ska vi uppdatera nätverket?
update_after_actions = 4
# Hur ofta ska Q-nätverket bli klonat från vårt nätverk
update_target_network = 10_000
# Vi använder HUBER för stabilitet, i kombo med Adam
loss_function = keras.losses.Huber()

In [ ]:
while True:
    observation, _ = env.reset()
    state = np.array(observation)
    episode_reward = 0
    
    for timestep in range(1, max_steps_per_episode):
        frame_count += 1
        
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = env.action_space.sample()
        else:
            state_tensor = keras.ops.convert_to_tensor(state)
            state_tensor = keras.ops.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            # Take best action
            action = keras.ops.argmax(action_probs[0].numpy())
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)
        
        state_next, reward, done, truncated, info = env.step(action)
        state_next = np.array(state_next)
        
        episode_reward += reward
        
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next
        
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
            indices = np.random.choice(range(len(done_history)), size=batch_size)
            
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = keras.ops.convert_to_tensor([float(done_history[i]) for i in indices])
            
            future_rewards = model_target.predict(state_next_sample, verbose=0)
            
            updated_q_values = rewards_sample + gamma * keras.ops.amax(future_rewards, axis=1)
            
            updated_q_values = updated_q_values*(1-done_sample) - done_sample
        
            masks = keras.ops.one_hot(action_sample, num_actions)
            
            with tf.GradientTape() as tape:
                q_values = model(state_sample)
                q_action = keras.ops.sum(tf.math.multiply(q_values, masks), axis=1)
                loss = loss_function(updated_q_values, q_action)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            
        if frame_count % update_target_network == 0:
            model_target.set_weights(model.get_weights())
            print(f"Best score of last 100 episodes: {max(episode_reward_history)}, episode: {episode_count}, frame: {frame_count}")
            model.save(f"breakout_qmodel_{episode_count}.keras")
        
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]
            
        if done:
            break
    
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)
    episode_count += 1
    if running_reward > 40:
        print(f"Solved at episode {episode_count}!")
        model.save(f"breakout_qmodel_solved.keras")
        break
    if (max_episodes > 0 and episode_count >= max_episodes):
        print(f"Max episodes reached")
        print(f"Stopped at episode {episode_count}")
        break
    if (max_frames > 0 and frame_count >= max_frames):
        print(f"Max frames reached")
        print(f"Stopped at frame {frame_count}")
        break

Best score of last 100 episodes: 8.0, episode: 49, frame: 10000


KeyboardInterrupt: 

In [ ]:
# observation, info = env.reset()

# episode_over = False

# while not episode_over:
#     action = env.action_space.sample()
#     observation, reward, terminated, truncated, info = env.step(action)
    
#     episode_over = terminated or truncated
    
# env.close() # För att frigöra minne